In [ ]:
#scrape jobs from ai-jobs.net
# import libraries
import requests
from bs4 import BeautifulSoup
import scrapy
import json
import re

In [ ]:
# get the html
ai_jobs_url = 'https://ai-jobs.net/'
ai_jobs_response = requests.get(ai_jobs_url)
ai_jobs_soup = BeautifulSoup(ai_jobs_response.content, 'html5lib')

print(ai_jobs_soup.prettify())

In [ ]:
# get the job list stored in ul tag with job-list id
job_list = ai_jobs_soup.find('ul', attrs = {'id': 'job-list'})

# get the job list items stored in li tags as next children of job_list
job_list_items = job_list.findChildren('li', recursive = False)

# get the job details from each job list item
job_details = []

# parse job description from apply link
def parse_job_description_ai_jobs(url):
    aijobs_job_page = requests.get(url)
    aijobs_job_page_soup = BeautifulSoup(aijobs_job_page.content, 'html5lib')
    job_description = aijobs_job_page_soup.find('div', attrs = {'id': 'job-description'}).decode_contents()
    return job_description

for job_list_item in job_list_items:
    job_detail = {}
    # get the job title stored in h3 tag
    job_detail['title'] = job_list_item.find('h3').text

    # store div.float-end as a variable
    float_end = job_list_item.find('div', attrs = {'class': 'float-end'})

    # get the job type stored in span tag with tex-bg-background inside div tag with class float-end
    job_detail['type'] = float_end.find('span', attrs = {'class': 'text-bg-secondary'}).text

    # get the job location stored in span tag with d-md-block inside div tag with class float-end
    job_detail['location'] = float_end.find('span', attrs = {'class': 'd-md-block'}).text

    # get all the div tag with class d-block

    div_d_block = job_list_item.find_all('div', attrs = {'class': 'd-block'})
    # get the count of div tags with class float-end
    div_count = len(div_d_block)
    job_salary = div_d_block[div_count - 1].find('span', attrs = {'class': 'text-bg-success'})

    # check if job_salary is not None
    if job_salary is not None:
        # print(div_count, job_salary.text)

        # get the salary_start and salary_end from the job_salary string stored in this format -> USD 80K - 110K
        # remove the USD/GBP or three lettered currency using regex from the string
        job_salary = re.sub(r'[A-Z]{3}', '', job_salary.text)

        # split the string by 'K' and get the first element for salary_start with space removed
        job_detail['salary_start'] = int(job_salary.split('K')[0].strip())

        # check if salary_end is present in the string by checking if the string contains '-'
        if '-' in job_salary:
            # split the string by '-' and get the second element for salary_end
            job_detail['salary_end'] = int(job_salary.split('-')[1].split('K')[0].strip())
        else:
            job_detail['salary_end'] = int(job_detail['salary_start'])

        # print(job_detail['salary_start'], job_detail['salary_end'])
    else:
        job_detail['salary_start'] = 00
        job_detail['salary_end'] = 00

    # get the job link stored in a tag with title attribute
    job_detail['website_link'] = job_list_item.find('a', attrs = {'title': 'View details for this job'})['href']

    # get the external job link from website_link
    # /job/42991-ai-content-creator/ -> https://ai-jobs.net/J42991/apply/
    job_detail['apply_link'] = 'https://ai-jobs.net' + job_detail['website_link'].replace('job/', 'J').split('-')[0] + '/apply/'
    # can't get the job description from the website
    job_detail['description'] = ''

    print(job_detail['description'])
    # tags for the job 
    # get all the span.text-bg-light tags upto 5th index
    tags = job_list_item.find_all('span', attrs = {'class': 'text-bg-light'})[:5]
    job_detail['tags'] = [i.text for i in tags]

    job_details.append(job_detail)


In [ ]:
# replace the job description with the parsed job description
for job in job_details:
    job['description'] = parse_job_description_ai_jobs('https://ai-jobs.net'+ job_detail['website_link'])
    

In [ ]:
# write the job details in a json file
with open('ai-jobs.json', 'w') as f:
    json.dump(job_details, f, indent = 4)

Get MachineHack jobs

In [ ]:
# await fetch("https://bootcamp.prod.machinehack.com/jobs/getAllJobs?current_page=1", {
#     "credentials": "omit",
#     "headers": {
#         "User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/109.0",
#         "Accept": "application/json, text/plain, */*",
#     },
#     "referrer": "https://machinehack.com/",
#     "method": "GET",
# });

# equivalent python code for the above fetch request
machinehack_url = 'https://bootcamp.prod.machinehack.com/jobs/getAllJobs?current_page=1'
machinehack_response = requests.get(machinehack_url, headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/109.0', 'Accept': 'application/json, text/plain, */*'})
machinehack_response = machinehack_response.json()


In [ ]:
# {
# 	"0": {
# 		"_id": "63e6346c3b5cfc0adbefe0b7",
# 		"experienceRequired": {
# 			"start": 3,
# 			"end": 5
# 		},
# 		"AnnualSalaryRange": {
# 			"start": 0
# 		},
# 		"timestamps": {
# 			"created_at": "2023-02-10T12:11:24.000Z",
# 			"updated_at": "2023-02-10T12:15:56.000Z"
# 		},
# 		"skills": [
# 			"Risk",
# 			"Modeling",
# 			"Credit Risk",
# 			"ECL",
# 			"Validation"
# 		],
# 		"location": [
# 			"Bangalore Urban"
# 		],
# 		"publishedAt": "2023-02-10T12:15:56.000Z",
# 		"questions": [],
# 		"jobTitle": "Data Scientist",
# 		"jobDescription": "<p>Credit Risk Model Developer</p><p>&nbsp;</p><p><strong>Experience: 3-5Y Responsibilities:</strong></p><p>&nbsp;</p><p>• Build, validate, document, implement and rebuild:<br>o Credit risk models (retail loan origination models, business banking customer rating</p><p>models, and loan behaviour scorecards)<br>o Collective Provision and Expected Loss methodologies. This includes all inputs of</p><p>Probability of Default, Loss Given Default and Exposure at Default (methodology).&nbsp;</p><p>&nbsp;</p><p>• Conduct detailed analytical work with a high level of accuracy in order to deliver high level</p><p>results to senior management and contribute to the management and education of enhanced credit risk approaches.</p><p>o Develop on going improvements to the model reporting.</p><p>o Responsible for managing issues through to resolution.</p><p>&nbsp;</p><p>• Define and specify key data requirements to support modelling approaches.</p><p>• Document model “technical manual”, modelling choices made, and model methodology</p><p>considerations.</p><p>• Engage with operational risk advisory, quantitative analyst, reporting and regulatory</p><p>specialist stakeholders.</p><p>• Assist in the development of other Credit Risk Modelling team members.</p><p>&nbsp;</p><p>• Working with credit risk Model Development Leaders to:</p><p>o Proactively engage with stakeholders to understand business context, add value, propose solutions, project manage pieces of work through to completion.</p><p>o Balance prioritization effectively between across a wide stakeholder group. Ensure time spent matches the importance of the work and manage situations / competing priorities so that the most beneficial program of work is achieved.</p><p>&nbsp;</p><p>• Working with the leaders of the Credit Risk Modelling team to ensure:<br>o Models are effectively embedded into operational activities<br>o The program of work for the department is documented and resourcing or delivery</p><p>issues are well managed.</p><p>&nbsp;</p><p>• Prepare and review analysis papers and ensure that high quality analytical papers are</p><p>written and delivered to appropriate senior management and committees. Present these</p><p>papers in an effective manner appropriate to the audience.</p><p>&nbsp;</p><p>• Analyse and constructively critique output across the wider Banking Credit function for the</p><p>business impacts (including regulatory and external audit) to ensure committee papers consider portfolio, modelling and data risks, and ultimately propose strategic recommendations that are underpinned by a compelling case.</p><p>&nbsp;</p><p>• Identifying inefficiencies and proposing operational process improvements to enable better outcomes.</p><p>• Add value to deliverables with excellent problem solving, idea generation and strategic thinking. Work closely with the wider Advanced Basel Project team, Banking Credit and other senior managers and Team Leaders. Active engagement in discussions with senior management to optimize the best solution for the bank and group.</p>",
# 		"slug": "data-scientist-da974",
# 		"jobType": "Permanent",
# 		"workType": "Hybrid",
# 		"postedBy": {
# 			"_id": "627b3f6a23e5b51a29d5cc4b",
# 			"firstName": "Sufiyan",
# 			"lastName": "S"
# 		},
# 		"companyId": {
# 			"_id": "6152a94e0ad6636dbf06081b",
# 			"companyName": "AIM Recruits",
# 			"timestamps": {
# 				"created_at": "2021-09-28T05:34:06.655Z",
# 				"updated_at": "2022-05-19T03:59:32.233Z"
# 			},
# 			"logo": "https://machinehack-enterprise-prod.s3.ap-south-1.amazonaws.com/machine_hack/production%20files/Machine_Hack__PLNNY__uploaded_at_1652687008014.jpeg",
# 			"website": "https://recruits.analyticsindiamag.com/"
# 		},
# 		"views": 1,
# 		"applicants": 0
# 	}
# }

# reformat above json to job_details array
job_details = []
for job in machinehack_response['result']['data']:

    job_details.append({
        'id': job['_id'],
        'title': job['jobTitle'],
        'type': job['jobType'] if 'jobType' in job else 'Not mentioned',
        'location': job['location'],
        'salary_start': job['AnnualSalaryRange']['start'],
        # if end salary is not mentioned, set it to start salary
        'salary_end': job['AnnualSalaryRange']['end'] if 'end' in job['AnnualSalaryRange'] else job['AnnualSalaryRange']['start'],
        'apply_link': 'https://machinehack.com/jobs/?currentJobId=' + job['_id'],
        'description': job['jobDescription'],
        'tags': job['skills'][:5],
    })


In [ ]:
# write .json results into a file
with open('machinehack_jobs.json', 'w') as f:
    json.dump(job_details, f, indent = 4)

Get mlconf jobs

In [ ]:
# await fetch("https://mlconf.com/jm-ajax/get_listings/", {
#     "headers": {
#         "User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/109.0",
#         "Accept": "*/*",
#         "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8"
#     },
#     "referrer": "https://mlconf.com/jobs/",
#     "body": "lang=&search_keywords=&search_location=&per_page=534&orderby=featured&order=DESC&page=1&show_pagination=false&form_data=search_keywords%3D%26search_location%3D",
#     "method": "POST",
# })

# equivalent to above fetch request
# max_jobs = 534  // at current time, would certainly change in future
# max number of jobs found out by passing 1 as value and response gives total number of pages which is 534

# create a function to get jobs from a page
def get_jobs(jobs_count):
    mlconf_response = requests.post('https://mlconf.com/jm-ajax/get_listings/', data = {
        'lang': '',
        'search_keywords': '',
        'search_location': '',
        'per_page': jobs_count, 
        'orderby': 'featured',
        'order': 'DESC',
        'page': 1,
        'show_pagination': 'false',
        'form_data': 'search_keywords=&search_location='
    })
    return mlconf_response.json()

# get jobs count from response by reading max_num_pages key from response when per_page is 1
jobs_count_current = get_jobs(1)['max_num_pages']
print(jobs_count_current)
# get all jobs with jobs_count
mlconf_response = get_jobs(jobs_count_current)

print(mlconf_response)

In [ ]:
# parse out html from response
soup = BeautifulSoup(mlconf_response['html'], 'html5lib')
print(soup.prettify())

In [ ]:
# reformat above html to separate out each job
# select all li tags with class job_listing
mlconf_job_listings = soup.select('li.type-job_listing')
# print(mlconf_job_listings)

job_details = []
for job in mlconf_job_listings:
    # get job title from h3 tag
    job_title = job.find('h3').text.strip()
    # get job type from class name among multiple classes of li tag job-type-*
    # <li class="post-15783 job_listing type-job_listing status-publish has-post-thumbnail entry job-type-full-time" data-latitude="" data-longitude="">
    # for above example, job type is full-time
    # get all classes of li tag
    li_classes = job['class']
    # get job type class from li_classes
    job_type = [type_class for type_class in li_classes if 'job-type-' in type_class]
    
    # if job type is not mentioned, set it to 'Not mentioned'
    if len(job_type) == 0:
        job_type = 'Not mentioned'
    else:
        # get job type from job_type list
        job_type = job_type[0].split('-')[2]

    # get job location from div tag with class location
    job_location = job.find('div', class_ = 'location').text.strip()
    # doesn't have salary info
    job_salary_start = 00
    job_salary_end = 00
    # get job apply link from a tag
    job_apply_link = job.find('a')['href']
    # Doesn't have job description
    job_description = ''
    # Doesn't have tags/skills/keywords
    job_tags = []

    job_details.append({
        'title': job_title,
        'type': job_type,
        'location': job_location,
        'salary_start': job_salary_start,
        'salary_end': job_salary_end,
        'website_link': job_apply_link,
        'description': job_description,
        'tags': job_tags,
    })


In [ ]:
# define a function to get jobs from a page
def get_job_description_mlconf(url):
    mlconf_response = requests.get(url)
    mlconf_soup = BeautifulSoup(mlconf_response.content, 'html5lib')
    # get job description from div tag with class job_description
    job_description = mlconf_soup.find('div', class_ = 'job_description').decode_contents()
    job_link = mlconf_soup.find('div', class_ = 'application_details').findChild('a')['href']
    return job_description, job_link


In [ ]:
# get jobs description from job apply link
# store failure as jobs started expiring in exception
count = 0
# iterate over job_details and get job description from apply link if count < 5
for job in job_details:
    if count < 5:
        try:
            job['description'], job['apply_link'] = get_job_description_mlconf(job['website_link'])
        except:
            job['description'] = ''
            count += 1
    else:
        break

In [ ]:
# write .json results into a file
with open('mlconf_jobs.json', 'w') as f:
    json.dump(job_details, f, indent = 4)
    

Get jobs from datajobs.com

In [ ]:
# get response from initial html response
datajobs_url = 'https://datajobs.com/Data-Science-Jobs'
datajobs_response = requests.get(datajobs_url)

# parse out html from response
soup = BeautifulSoup(datajobs_response.content, 'html5lib')
# print(soup.prettify())

In [ ]:
# get div tag with style attribute and value as 'display: table-cell'
# this consists of all job listings
datajobs_job_listings = soup.find('div', attrs = {'style': 'display:table-cell; width:520px;'})
# print(datajobs_job_listings.prettify())

# get all div tags within datajobs_job_listings nonrecursively leaving out last div tag
datajobs_job_listings = datajobs_job_listings.find_all('div', recursive = False)[:-1]

job_details = []

# get all job details from datajobs_job_listings
for job in datajobs_job_listings:
    # get a tag within job div
    job_a_tag = job.find('a')
    # get job title from strong tag within a tag
    job_title = job_a_tag.find('strong').text.strip()
    # doesn't have job type
    job_type = 'Not mentioned'
    # get location from em tag within job div
    job_location = job.find('em').text.strip().split('\n')[0]
    # check if salary is mentioned in job em tag
    if '\u2013' in job.find('em').text.strip():
        # get salary from job_location
        job_salary_start = int(job.find('em').text.split('$')[1].split('\u2013')[0].strip().split(',')[0])
        # print("job_salary_start", job_salary_start)
        job_salary_end = int(job.find('em').text.split('$')[2].split('\n')[0].strip().split(',')[0])
        # get end salary if mentioned
    else:
        # doesn't have salary info
        job_salary_start = 00
        job_salary_end = 00

    # get job apply link from href attribute of a tag
    website_link = "https://datajobs.com" + job_a_tag['href']

    # create id for job
    job_id = website_link.split('~')[-1]

    # doesn't have job description
    job_description = ''
    # doesn't have tags/skills/keywords
    job_tags = []

    job_details.append({
        'id': job_id,
        'title': job_title,
        'type': job_type,
        'location': job_location,
        'salary_start': job_salary_start,
        'salary_end': job_salary_end,
        'website_link': website_link,
        'apply_link': '',
        'description': job_description,
        'tags': job_tags,
    })

In [ ]:
# get job description from job apply link
def get_job_description_datajobs(url):
    datajobs_response = requests.get(url)
    datajobs_soup = BeautifulSoup(datajobs_response.content, 'html5lib')
    # get job description from first div tag with class jobpost-table-cell-2
    job_description = datajobs_soup.find('div', class_ = 'jobpost-table-cell-2').decode_contents()
    return job_description

In [ ]:
# get job apply link from website link
def get_job_apply_link_datajobs(job_id):
    # create request url for apply link from job id
    url = 'https://datajobs.com/job-posting-applyto-ajax?job_id='+job_id
    # get response from request url
    datajobs_response = requests.get(url)
    # parse out html from response
    datajobs_soup = BeautifulSoup(datajobs_response.content, 'html5lib')
    # get job apply link from href attribute of a tag
    print(datajobs_soup.prettify(), job_id, url)
    job_apply_link = datajobs_soup.find('a')['href']
    return job_apply_link

In [ ]:
for job in job_details:
    job['description'] = get_job_description_datajobs(job['website_link'])
    job['apply_link'] = get_job_apply_link_datajobs(job['id'])

In [ ]:
# write .json results into a file
with open('datajobs_jobs.json', 'w') as f:
    json.dump(job_details, f, indent = 4)